In [1]:
import sys
import math
from PyQt6.QtWidgets import QApplication, QGraphicsView, QGraphicsScene, QGraphicsPixmapItem, QVBoxLayout, QWidget
from PyQt6.QtGui import QPixmap, QPen, QColor, QTransform, QPainter, QBrush
from PyQt6.QtCore import Qt, QRectF, QPointF, QSizeF  # Import QSizeF

class CustomGraphicsView(QGraphicsView):
    def __init__(self):
        super().__init__()
        self.scene = QGraphicsScene()
        self.setScene(self.scene)
        self.main_pixmap_item = None
        self.original_pixmap = None
        self.start_pos = None
        self.current_rect = None
        self.selection_rect = None
        self.selected_pixmap_item = None
        self.dragging = False
        self.rotating = False
        self.last_angle = 0

        # Enable transparency
        self.setAttribute(Qt.WidgetAttribute.WA_TranslucentBackground)
        self.viewport().setAttribute(Qt.WidgetAttribute.WA_TranslucentBackground)
        self.setBackgroundBrush(QBrush(QColor(0, 0, 0, 0)))  # Make the view background transparent
        self.scene.setBackgroundBrush(QBrush(QColor(0, 0, 0, 0)))  # Make the scene background transparent
        self.setRenderHint(QPainter.RenderHint.Antialiasing)
        self.setRenderHint(QPainter.RenderHint.SmoothPixmapTransform)

    def load_image(self, image_path):
        try:
            self.original_pixmap = QPixmap(image_path)
            if self.original_pixmap.isNull():
                raise ValueError(f"Could not load image from {image_path}")
            self.main_pixmap_item = QGraphicsPixmapItem(self.original_pixmap)
            self.scene.addItem(self.main_pixmap_item)
            self.setSceneRect(self.main_pixmap_item.boundingRect())
        except FileNotFoundError:
            print(f"Error: Image file not found at {image_path}")
        except ValueError as e:
            print(f"Error: {e}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

    def mousePressEvent(self, event):
        pos = self.mapToScene(event.pos())
        if event.button() == Qt.MouseButton.LeftButton:
            if self.selected_pixmap_item:
                self.dragging = True
                self.drag_start = pos
            else:
                self.start_pos = pos
                self.current_rect = None
        elif event.button() == Qt.MouseButton.RightButton and self.selected_pixmap_item:
            self.rotating = True
            self.rotation_center = self.selected_pixmap_item.sceneBoundingRect().center()
            self.last_angle = self.angle_between_points(self.rotation_center, pos)
        super().mousePressEvent(event)

    def mouseMoveEvent(self, event):
        pos = self.mapToScene(event.pos())
        if self.dragging and self.selected_pixmap_item:
            delta = pos - self.drag_start
            self.selected_pixmap_item.moveBy(delta.x(), delta.y())
            self.drag_start = pos
        elif self.rotating and self.selected_pixmap_item:
            new_angle = self.angle_between_points(self.rotation_center, pos)
            delta_angle = new_angle - self.last_angle
            self.selected_pixmap_item.setRotation(self.selected_pixmap_item.rotation() + delta_angle)
            self.last_angle = new_angle
        elif self.start_pos and not self.selected_pixmap_item:
            self.current_rect = QRectF(self.start_pos, pos).normalized()
            if self.selection_rect:
                self.scene.removeItem(self.selection_rect)
            self.selection_rect = self.scene.addRect(self.current_rect, QPen(QColor(255, 0, 0), 2, Qt.PenStyle.DashLine))
        super().mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.MouseButton.LeftButton:
            self.dragging = False
            if self.current_rect and not self.selected_pixmap_item:
                self.cut_and_create_selected_pixmap()
            self.start_pos = None
        elif event.button() == Qt.MouseButton.RightButton:
            self.rotating = False
        super().mouseReleaseEvent(event)

    def mouseDoubleClickEvent(self, event):
        self.apply_transformations()

        # Reset for next selection
        self.selected_pixmap_item = None  # Very important: Allow new selection
        self.start_pos = None
        self.current_rect = None
        self.selection_rect = None
        self.dragging = False
        self.rotating = False
        self.update()
        super().mouseDoubleClickEvent(event)

    def wheelEvent(self, event):
        if self.selected_pixmap_item:
            scale_factor = 1.1 if event.angleDelta().y() > 0 else 0.9
            self.selected_pixmap_item.setScale(self.selected_pixmap_item.scale() * scale_factor)
        super().wheelEvent(event)

    def cut_and_create_selected_pixmap(self):
        if self.current_rect:
            rect = self.current_rect.toRect()
            selected_pixmap = self.original_pixmap.copy(rect)

            # Create the selected pixmap item
            if self.selected_pixmap_item:
                self.scene.removeItem(self.selected_pixmap_item)
            self.selected_pixmap_item = QGraphicsPixmapItem(selected_pixmap)
            self.selected_pixmap_item.setPos(self.current_rect.topLeft())
            self.scene.addItem(self.selected_pixmap_item)

            # Remove the selection rectangle
            if self.selection_rect:
                self.scene.removeItem(self.selection_rect)
            self.selection_rect = None

            # Create a new pixmap for the main image with a transparent hole
            new_main_pixmap = QPixmap(self.original_pixmap.size())
            new_main_pixmap.fill(Qt.GlobalColor.transparent)
            painter = QPainter(new_main_pixmap)
            painter.setCompositionMode(QPainter.CompositionMode.CompositionMode_Source)
            painter.drawPixmap(0, 0, self.original_pixmap)
            painter.setCompositionMode(QPainter.CompositionMode.CompositionMode_Clear)
            painter.fillRect(rect, QColor(0, 0, 0, 255))
            painter.end()

            # Update the main pixmap item
            self.main_pixmap_item.setPixmap(new_main_pixmap)
            self.original_pixmap = new_main_pixmap  # Update the original pixmap
            self.start_pos = None  # Reset the start position after cutting
            self.update()

    def reset_image(self):
        if self.original_pixmap:
            self.main_pixmap_item.setPixmap(self.original_pixmap)
        if self.selected_pixmap_item:
            self.scene.removeItem(self.selected_pixmap_item)
        if self.selection_rect:
            self.scene.removeItem(self.selection_rect)
        self.selected_pixmap_item = None
        self.selection_rect = None
        self.current_rect = None
        self.dragging = False
        self.rotating = False
        self.update()

    def angle_between_points(self, center, point):
        delta = point - center
        return math.degrees(math.atan2(delta.y(), delta.x()))

    def apply_transformations(self):
        if self.selected_pixmap_item:
            # Get the transformed pixmap
            transformed_pixmap = self.get_transformed_pixmap()

            # Determine the destination rectangle in the original pixmap
            dest_rect = QRectF(self.selected_pixmap_item.pos(), QSizeF(transformed_pixmap.width(), transformed_pixmap.height())).toRect()

            # Create a new pixmap to hold the modified original image
            new_main_pixmap = QPixmap(self.original_pixmap.size())
            new_main_pixmap.fill(Qt.GlobalColor.transparent)

            # Create a painter to draw on the new pixmap
            painter = QPainter(new_main_pixmap)
            painter.setCompositionMode(QPainter.CompositionMode.CompositionMode_Source)
            painter.drawPixmap(0, 0, self.original_pixmap)

            # Clear the area where the transformed pixmap will be pasted
            painter.setCompositionMode(QPainter.CompositionMode.CompositionMode_Clear)
            painter.fillRect(dest_rect, QColor(0, 0, 0, 255))

            # Paste the transformed pixmap
            painter.setCompositionMode(QPainter.CompositionMode.CompositionMode_SourceOver)
            painter.drawPixmap(dest_rect.topLeft(), transformed_pixmap)

            painter.end()

            # Update the main pixmap item with the modified image
            self.main_pixmap_item.setPixmap(new_main_pixmap)
            self.original_pixmap = new_main_pixmap

            # Remove the selected pixmap item
            self.scene.removeItem(self.selected_pixmap_item)
            self.selected_pixmap_item = None
            self.update()

    def get_transformed_pixmap(self):
        # Create a new pixmap
        new_pixmap = QPixmap(self.selected_pixmap_item.pixmap().size())
        new_pixmap.fill(Qt.GlobalColor.transparent)  # Ensure transparency

        # Create a painter for the new pixmap
        painter = QPainter(new_pixmap)

        # Set the transform based on the selected item
        transform = QTransform()
        transform.translate(self.selected_pixmap_item.pixmap().rect().width() / 2,
                            self.selected_pixmap_item.pixmap().rect().height() / 2)
        transform.rotate(self.selected_pixmap_item.rotation())
        transform.scale(self.selected_pixmap_item.scale(), self.selected_pixmap_item.scale())
        transform.translate(-self.selected_pixmap_item.pixmap().rect().width() / 2,
                            -self.selected_pixmap_item.pixmap().rect().height() / 2)
        painter.setTransform(transform)

        # Draw the original pixmap onto the new pixmap
        painter.drawPixmap(0, 0, self.selected_pixmap_item.pixmap())
        painter.end()

        return new_pixmap

class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.init_ui()
        # Make the main window transparent
        self.setAttribute(Qt.WidgetAttribute.WA_TranslucentBackground)
        self.setStyleSheet("background: transparent;")  # Ensure the main window's background is also transparent

    def init_ui(self):
        layout = QVBoxLayout()
        self.view = CustomGraphicsView()
        layout.addWidget(self.view)
        self.setLayout(layout)
        self.setWindowTitle("Image Selection Tool")
        self.resize(800, 600)
        # Load the image with the provided path
        self.view.load_image("images/test/2_people_together.png")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    # Set application-level style sheet for transparency (important for some systems)
    app.setStyleSheet("QGraphicsView { background: transparent; border: none; }")
    window = MainWindow()
    window.show()
    sys.exit(app.exec())

SystemExit: 0

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
